In [1]:
# Install PyTorch 2.1.2 (compatible with Triton 2.1.0 and BitsAndBytes)
!pip install -q torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121



# Install data/model libraries
!pip install -q pyarrow==19.0.1 datasets==2.20.0

# Install transformers + training libraries
!pip install -q transformers==4.40.2 tokenizers==0.19.1
!pip install -q trl==0.9.4 peft==0.10.0 accelerate==0.28.0 bitsandbytes==0.43.1

print("✅ All packages installed")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 400.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 95.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 22.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
s3fs 2025.3.0 requires fsspec==2025.3.0.*

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

bnb = BitsAndBytesConfig(load_in_4bit=True)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    quantization_config=bnb,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/fine-tune-phi/other/default/1")

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "/kaggle/input/fine-tune-phi/other/default/1")

model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The repository for microsoft/Phi-3-mini-4k-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3-mini-4k-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository for microsoft/Phi-3-mini-4k-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3-mini-4k-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
             

In [6]:
prompt = "<|user|>Top 5 products by total ordered quantity.\n<|end|>\n<|assistant|>\n"

tokens = tokenizer(prompt, return_tensors="pt").to(model.device)
output = model.generate(**tokens, max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Top 5 products by total ordered quantity.
 MATCH (p:Product)<-[:ORDERS]-(o:CustomerOrder) WITH p, sum(o.quantity) AS qty RETURN p.name, qty ORDER BY qty DESC LIMIT 5;


In [7]:
!pip install transformers==4.40.2 peft==0.10.0 accelerate bitsandbytes
!apt-get -y install git-lfs


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 165 not upgraded.


In [9]:
LORA_DIR = "/kaggle/input/fine-tune-phi/other/default/1"    # your uploaded folder
BASE_MODEL = "microsoft/Phi-3-mini-4k-instruct"
MERGED_DIR = "/kaggle/working/phi3_merged_fp16"


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

print("Loading base model...")
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="cpu",
    trust_remote_code=True
)

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)

print("Loading LoRA adapter:", LORA_DIR)
peft_model = PeftModel.from_pretrained(
    base,
    LORA_DIR,
    torch_dtype=torch.float16
)

print("Merging LoRA → base model...")
merged = peft_model.merge_and_unload()

print("Saving merged FP16 model to:", MERGED_DIR)
merged.save_pretrained(MERGED_DIR)
tokenizer.save_pretrained(MERGED_DIR)

print("Merge complete.")


Loading base model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading LoRA adapter: /kaggle/input/fine-tune-phi/other/default/1
Merging LoRA → base model...
Saving merged FP16 model to: /kaggle/working/phi3_merged_fp16
Merge complete.


In [32]:
!git clone https://github.com/ggerganov/llama.cpp /kaggle/working/llama.cpp


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into '/kaggle/working/llama.cpp'...
remote: Enumerating objects: 70042, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 70042 (delta 155), reused 96 (delta 96), pack-reused 69782 (from 2)
Receiving objects: 100% (70042/70042), 214.40 MiB | 33.72 MiB/s, done.
Resolving deltas: 100% (50610/50610), done.


In [33]:
!ls /kaggle/working/llama.cpp



AUTHORS			       docs	   pocs
benches			       examples    poetry.lock
build-xcframework.sh	       flake.lock  pyproject.toml
ci			       flake.nix   pyrightconfig.json
cmake			       ggml	   README.md
CMakeLists.txt		       gguf-py	   requirements
CMakePresets.json	       grammars    requirements.txt
CODEOWNERS		       include	   scripts
common			       LICENSE	   SECURITY.md
CONTRIBUTING.md		       licenses    src
convert_hf_to_gguf.py	       Makefile    tests
convert_hf_to_gguf_update.py   media	   tools
convert_llama_ggml_to_gguf.py  models	   vendor
convert_lora_to_gguf.py        mypy.ini


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [39]:
!ls -lh /kaggle/working


total 12K
drwxr-xr-x 25 root root 4.0K Dec  1 20:29 llama.cpp
-rw-r--r--  1 root root   22 Dec  1 20:33 phi3_gguf_q4.zip
drwxr-xr-x  2 root root 4.0K Dec  1 20:13 phi3_merged_fp16


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
!python3 /kaggle/working/llama.cpp/convert_hf_to_gguf.py \
    /kaggle/working/phi3_merged_fp16 \
    --outfile /kaggle/working/phi3_gguf_q8_0.gguf \
    --outtype q8_0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO:hf-to-gguf:Loading model: phi3_merged_fp16
INFO:hf-to-gguf:Model architecture: Phi3ForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00002.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00002.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:output.weight,             torch.float16 --> Q8_0, shape = {3072, 32064}
INFO:hf-to-gguf:blk.21.attn_norm.weight,   torch.float16 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.21.ffn_down.weight,    torch.float16 --> Q8_0, shape = {8192, 3072}
INFO:hf-to-gguf:blk.21.ffn_up.weight,      torch.float16 --> Q8_0, shape = {3072, 16384}
INFO:hf-to-gguf:blk.21.ffn_norm.weight,    torch.float16 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.21.attn_qkv.weight,    torch.float16 --> Q8_0, shape = {3072, 9216}
INFO:hf-to-gguf:blk.22.attn_norm.weight,   

In [51]:
import shutil

zip_path = "/kaggle/working/phi3_gguf_q8_0.zip"

shutil.make_archive(zip_path.replace(".zip",""), "zip", "/kaggle/working", "phi3_gguf_q8_0.gguf")

print("ZIP created:", zip_path)


ZIP created: /kaggle/working/phi3_gguf_q8_0.zip


In [52]:
from IPython.display import FileLink
FileLink('/kaggle/working/phi3_gguf_q8_0.zip')


/kaggle/working/phi3_gguf_q8_0.zip